<a href="https://colab.research.google.com/github/Sasidhar0709/Project-Stock-market-Analysis/blob/main/Project_Final_21063192.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import yfinance as yf
from scipy.interpolate import interp1d
from scipy.stats import linregress
import warnings
warnings.filterwarnings('ignore')

In [ ]:
ftse100_tickers = [
    'AAF.L', 'AAL.L', 'ABF.L', 'ADM.L', 'AHT.L', 'ANTO.L', 'AUTO.L', 'AV.L',
    'AZN.L', 'BA.L', 'BARC.L', 'BATS.L', 'BDEV.L', 'BEZ.L', 'BKG.L', 'BME.L',
    'BNZL.L', 'BP.L', 'BRBY.L', 'BT-A.L', 'CCH.L', 'CNA.L', 'CPG.L', 'CRDA.L',
    'CTEC.L', 'DARK.L', 'DCC.L', 'DGE.L','DPLM.L', 'ENT.L', 'EXPN.L','EZJ.L',
    'FCIT.L', 'FRAS.L', 'FRES.L', 'GLEN.L', 'GSK.L', 'HIK.L','HL.L', 'HLMA.L',
    'HLN.L', 'HSBA.L','HWDN.L','IAG.L','ICG.L','IHG.L', 'III.L','IMB.L',
    'IMI.L','INF.L','ITRK.L', 'JD.L', 'KGF.L', 'LAND.L', 'LGEN.L', 'LLOY.L',
    'LMP.L', 'LSEG.L','MKS.L', 'MNG.L', 'MNDI.L', 'MRO.L', 'NG.L', 'NWG.L',
    'NXT.L', 'PHNX.L', 'PRU.L', 'PSH.L', 'PSN.L','PSON.L', 'REL.L', 'RIO.L',
    'RKT.L', 'RMV.L','RR.L','RTO.L', 'SBRY.L', 'SDR.L', 'SGE.L', 'SGRO.L',
    'SHEL.L','SKG.L', 'SMDS.L', 'SMIN.L', 'SMT.L', 'SN.L', 'SPX.L', 'SSE.L',
    'STAN.L', 'SVT.L', 'TSCO.L', 'TW.L', 'ULVR.L','UTG.L', 'UU.L','VOD.L',
    'VTY.L','WEIR.L', 'WPP.L', 'WTB.L'  ]


# Define the start and end date
start_date = '2013-01-01'
end_date = '2023-12-31'

# Initialize an empty DataFrame to hold all data
all_data = pd.DataFrame()

# Fetch the data for each ticker
for ticker in ftse100_tickers:
    print(f"Fetching data for {ticker}")
    try:
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        stock_data['Ticker'] = ticker  # Add a column for the ticker symbol
        all_data = pd.concat([all_data, stock_data])
    except Exception as e:
        print(f"Could not fetch data for {ticker}. Error: {e}")

# Reset index to have a column with the date
all_data.reset_index(inplace=True)

# Save the data to a CSV file
all_data.to_csv('ftse100_historical_data.csv', index=False)
print("Data has been saved to ftse100_historical_data.csv")


[*********************100%%**********************]  1 of 1 completed

Fetching data for AAF.L
Fetching data for AAL.L



[*********************100%%**********************]  1 of 1 completed


Fetching data for ABF.L


[*********************100%%**********************]  1 of 1 completed


...

Fetching data for WEIR.L


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Fetching data for WPP.L
Fetching data for WTB.L


[*********************100%%**********************]  1 of 1 completed


Data has been saved to ftse100_historical_data.csv


In [ ]:
df_allstocks = pd.read_csv('ftse100_historical_data.csv')

In [ ]:
df_allstocks = df_allstocks.pivot(index='Date', columns='Ticker', values='Adj Close')
df_allstocks.reset_index(inplace=True)
df_allstocks.columns.name = None
df_allstocks = df_allstocks.set_index('Date')

In [ ]:
df_allstocks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2778 entries, 2013-01-02 to 2023-12-29
Data columns (total 100 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAF.L   1137 non-null   float64
 1   AAL.L   2776 non-null   float64
 2   ABF.L   2778 non-null   float64
 3   ADM.L   2778 non-null   float64
 4   AHT.L   2778 non-null   float64
 5   ANTO.L  2778 non-null   float64
 6   AUTO.L  2218 non-null   float64
 7   AV.L    2778 non-null   float64
 8   AZN.L   2778 non-null   float64
 9   BA.L    2778 non-null   float64
 10  BARC.L  2777 non-null   float64
 11  BATS.L  2777 non-null   float64
 12  BDEV.L  2778 non-null   float64
 13  BEZ.L   2778 non-null   float64
 14  BKG.L   2778 non-null   float64
 15  BME.L   2414 non-null   float64
 16  BNZL.L  2778 non-null   float64
 17  BP.L    2771 non-null   float64
 18  BRBY.L  2778 non-null   float64
 19  BT-A.L  2778 non-null   float64
 20  CCH.L   2697 non-null   float64
 21  CNA.L   2778 non-null   fl

In [ ]:
df_allstocks.isnull().sum()

AAF.L     1641
AAL.L        2
ABF.L        0
ADM.L        0
AHT.L        0
          ... 
VOD.L        1
VTY.L        0
WEIR.L       1
WPP.L        1
WTB.L        1
Length: 100, dtype: int64

In [ ]:
df_allstocks.dropna(inplace=True,axis=1)

In [ ]:
df_allstocks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2778 entries, 2013-01-02 to 2023-12-29
Data columns (total 63 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ABF.L   2778 non-null   float64
 1   ADM.L   2778 non-null   float64
 2   AHT.L   2778 non-null   float64
 3   ANTO.L  2778 non-null   float64
 4   AV.L    2778 non-null   float64
 5   AZN.L   2778 non-null   float64
 6   BA.L    2778 non-null   float64
 7   BDEV.L  2778 non-null   float64
 8   BEZ.L   2778 non-null   float64
 9   BKG.L   2778 non-null   float64
 10  BNZL.L  2778 non-null   float64
 11  BRBY.L  2778 non-null   float64
 12  BT-A.L  2778 non-null   float64
 13  CNA.L   2778 non-null   float64
 14  DGE.L   2778 non-null   float64
 15  DPLM.L  2778 non-null   float64
 16  ENT.L   2778 non-null   float64
 17  FCIT.L  2778 non-null   float64
 18  FRAS.L  2778 non-null   float64
 19  FRES.L  2778 non-null   float64
 20  HL.L    2778 non-null   float64
 21  HLMA.L  2778 non-null   flo